In [47]:
import pyodbc
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import adodbapi

In [48]:
server = 'your server'
database = 'your database'
connection_string = f"Provider=MSOLAP;Data Source={server};Initial Catalog={database};"

In [49]:
scopes=['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
google_sheet_credentials = ServiceAccountCredentials.from_json_keyfile_name('key3.json', scopes)

In [50]:
try:
    conn = adodbapi.connect(connection_string)
    cursor = conn.cursor()
except adodbapi.Error as e:
    print("data connection error")
    exit()

In [51]:
#execute the queries 
try:
    cursor.execute("SELECT * FROM $SYSTEM.MDSCHEMA_MEASURES")
    md_schema_measures = cursor.fetchall()
except adodbapi.Error as e:
    print("Error executing SQL queries:", e)
    cursor.close()
    conn.close()
    exit()

In [52]:
# Connect to Google Sheets
try:
    gc = gspread.authorize(google_sheet_credentials)
    sheet = gc.open_by_url("https://docs.google.com/spreadsheets/your URL")
    worksheet = sheet.get_worksheet(0)  # Assuming you want to write to the first worksheet
except gspread.exceptions.APIError as e:
    print("Google Sheets API Error:", e)
    cursor.close()
    conn.close()
    exit()

In [53]:
# Write results to Google Sheets
try:
    # Clear existing data in the worksheet
    worksheet.clear()

    # Write MDSCHEMA_MEASURES to Google Sheets with header
    if cursor.description:
        headers = [description[0] for description in cursor.description]
        worksheet.insert_row(headers, index=1)  # Insert header row at index 1

    for i, row in enumerate(md_schema_measures):
        values = [str(value) for value in row]  # Convert values to strings if necessary
        worksheet.insert_row(values, index=i+2)  # Insert data rows below header

    print("Data successfully written to Google Sheets.")
except gspread.exceptions.APIError as e:
    print("Error writing data to Google Sheets:", e)


Error writing data to Google Sheets: {'code': 429, 'message': "Quota exceeded for quota metric 'Write requests' and limit 'Write requests per minute per user' of service 'sheets.googleapis.com' for consumer 'project_number:650650901881'.", 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'quota_limit': 'WriteRequestsPerMinutePerUser', 'quota_location': 'global', 'service': 'sheets.googleapis.com', 'quota_metric': 'sheets.googleapis.com/write_requests', 'consumer': 'projects/650650901881', 'quota_limit_value': '60'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quota#requesting_higher_quota'}]}]}
